In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
#import spectral
from sklearn import preprocessing
from sklearn.cluster import KMeans 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
#from xgboost import XGBClassifier
from sklearn.decomposition import PCA
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.ensemble import AdaBoostClassifier
#import lightgbm as lgb
import joblib

### 定义相关函数

In [2]:
## 降维操作
def pca_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    ## 降维
    pca = PCA(n_components=dimension,svd_solver='auto')
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    ## 姜维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,145)
    pca_return_array = np.array(pca_return_array)
    print (pca.explained_variance_ratio_)
    return pca_return_array
  
  # 归一化后降维
def pca_scal_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    spectral_array = preprocessing.scale(spectral_array) # 归一化
    pca = PCA(n_components=dimension,svd_solver='auto')
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    ## 姜维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,145)
    pca_return_array = np.array(pca_return_array)
    print (pca.explained_variance_ratio_)
    return pca_return_array 

# pca后降维
def pca_scal_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    ## 降维
    pca = PCA(n_components=dimension,svd_solver='auto')
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    # 归一化操作
    pca_input_iamge = preprocessing.scale(pca_input_iamge)
    ## 降维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,145)
    pca_return_array = np.array(pca_return_array)
    print (pca.explained_variance_ratio_)
    return pca_return_array

#####-------------------------------------------------- 生成 3D Tensor---------------------------########

# 可以提取出高光谱数据的一个channel的数据 145*145
def split_one_channel(A,channel):
    row = A.shape[0]
    d = [0]*row
    for i in range(row):
        d[i] = A[i][:,channel-1]
    e = np.vstack((d[i] for i in range(row)))
    return e   
# 参数说明
# channel 通道
# A 高光谱立方块


# 依据patch大小（Filter 大小）对矩阵进行镜像处理
def mirror_matrix(A, fil):
    pad = int((fil-1)/2)
    for  i in range(1,pad+1):
        up = np.array(A[2*i-1,:])
        down = np.array(A[-2*i,:])
        Matrix_up_down = np.vstack([up,A,down])
        left = np.array(Matrix_up_down[:,2*i-1]).reshape(len(Matrix_up_down),1)
        right = np.array(Matrix_up_down[:,-2*i]).reshape(len(Matrix_up_down),1)
        Matrix_left_right = np.hstack([left,Matrix_up_down,right,])
        A = Matrix_left_right
    return A

# 参数说明
# n   array size
# fil  filter size5*5
# pad = (fil-1)/2  padding大小


def recreate_whole_data(dim,fil,input_image):
    whole_origin_matrix_array = np.array([split_one_channel(input_image,i) for i in range(1,dim+1)])
    whole_mirror_matrix_array = np.array([mirror_matrix(whole_origin_matrix_array[i],fil) for i in range(dim)])
    return whole_mirror_matrix_array


## 分割patch  work out 一个维度的patch
def create_patch_one(A,fil):
    whole_arr = list()
    row = A.shape[0]-fil+1
    column = A.shape[1]-fil+1
    for i in range(row):
        for j in range(column):
            temp_arr = list()
            for z in range(fil):
                p = A[i+z,j:j+fil]
                temp_arr.append(p)
            one_arr = np.vstack(temp_arr)
            whole_arr.append(one_arr)
    return np.array(whole_arr)
    
def create_patch_whole(A,fil,dim):
    temp = [create_patch_one(split_one_channel(A,i),fil) for i in range(1,dim+1)]
    return np.array(temp)


def create_patch_cube(patch,fil,dim):
    temp = list()
    nums = patch[0].shape[0] # the nums of every dim of windows  n - f + 1
    for j in range(nums):
        temp_re = np.vstack([patch[i][j] for i in range(dim)]).reshape(dim,fil,fil)
        temp_re1= temp_re.transpose(2,1,0) # switch 1,3 dim
        temp_re2 = temp_re1.transpose(1,0,2) # switch 1,2 dim
        temp.append(temp_re2)
    patch_cubes = np.array(temp)
    return patch_cubes

def whole_sample_lable(patch_cubes):
    sample_lable_list = list()
    for i in range(len(patch_cubes)):
            sample_lable_list.append((patch_cubes[i],out_labels[i]))
    whole_sample_lable_array = np.array(sample_lable_list)
    return whole_sample_lable_array

# 提取有用的标签
def useful_sample_lable(patch_cubes):
    sample_lable_list = list()
    for i in range(len(patch_cubes)):
        if out_labels[i] != 0:
            sample_lable_list.append((patch_cubes[i],out_labels[i]))
    sample_lable_array = np.array(sample_lable_list)
    return sample_lable_array

  
# 存储numpy文件
def save_sample_label(sample_lable_array,path='H:\data\sample_lable_array'):
    np.save(path,sample_lable_array)
    
# 载入numpy文件
def load_sample_label(name):
    s = np.load(name)
    return s
  
  
# ## 将数据存储：
# def scaler_save(sample_lab_fix, path = 'H:\data\spec_spi.csv'):
#     ## 没有归一化
#     data_D = sample_lab_fix[:,:-1]
#     data_L = sample_lab_fix[:,-1]

#     ## 存储
#     new = np.column_stack((data_D, data_L))
#     new_ = pd.DataFrame(new)
#     new_.to_csv(path,header=False,index=False)

#     ## 分类数据提取
# def classify_pre(path='H:\data\spec_spi.csv'):
#     data = pd.read_csv(path,header=None)
#     data = data.as_matrix()
#     data_D = data[:,:-1]
#     data_L = data[:,-1]
#     data_train, data_test, label_train, label_test = train_test_split(data_D,data_L,test_size=0.9)
#     return data_train, data_test, label_train, label_test

### 数据预处理

In [3]:
DIM = 10
FIL = 5

In [4]:
inp = loadmat('F:\data\Indian_pines_corrected.mat')['indian_pines_corrected']
out = loadmat('F:\data\Indian_pines_gt.mat')['indian_pines_gt']

In [5]:
pca_return_array_alpha = pca_scal_hpi(inp,DIM) # pca

[0.68493799 0.23531371 0.01496355 0.00821542 0.00695011 0.00517011
 0.00399681 0.0036236  0.00307127 0.00293212]


In [6]:
pca_return_array_beta = recreate_whole_data(DIM,FIL,pca_return_array_alpha) # padd 并镜像

In [7]:
pca_return_array_beta.shape # 145+2*（f-1）/2

(10, 149, 149)

In [8]:
## 将pca_return beta 变为 149--3  x  149--2  x 30 形状
pca_return_array_beta = pca_return_array_beta.transpose(2,1,0) #switch dim 1,3

In [9]:
pca_return_array_beta.shape

(149, 149, 10)

In [10]:
pca_return_array_beta = pca_return_array_beta.transpose(1,0,2) # switch dim 1,2

In [11]:
pca_return_array_beta.shape

(149, 149, 10)

In [12]:
## ## 将每个 filxfil (5x5)的patch取出来， 并将每个channel的信息放入一个集合
patch_dim = create_patch_whole(pca_return_array_beta,FIL,DIM)

In [13]:
patch_dim.shape  # 30 个dim  every dim 21025个window (sample)    window(5x5)

(10, 21025, 5, 5)

In [14]:
patch_cubes = create_patch_cube(patch_dim,FIL, DIM)

In [15]:
patch_cubes.shape   # 21025个 sample 每个sample是一个patch 30dim  5x5

(21025, 5, 5, 10)

In [16]:
out_labels = out.reshape(145*145,)

In [17]:
sample_lable_array = useful_sample_lable(patch_cubes)

In [18]:
sample_lable_array.shape  # 10249个样本 ，2个values, 1st sample ,  2st lables

(10249, 2)

In [19]:
sample_lable_array[0][0].shape

(5, 5, 10)

In [20]:
save_sample_label(sample_lable_array,path='F:\data\sample_lable_array')

In [21]:
whole_data = load_sample_label('F:\data\sample_lable_array.npy')

In [22]:
whole_data.shape

(10249, 2)

In [23]:
Data = np.array([whole_data[:,0][i]for i in range(whole_data.shape[0])])  ## 取出数据

In [24]:
Data.shape

(10249, 5, 5, 10)

## 定义模型

In [25]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD
from keras.optimizers import Adam

model = Sequential()
# input : 5x5 patch with 30 channels - > (5,5,30)
# layer1 Conv2d  3 x 3 = 90 filters
model.add(Conv2D(10, (3, 3), activation='relu', input_shape=(5, 5, 10)))

model.add(Conv2D(20, (3, 3), activation='relu'))

model.add(Flatten())          
model.add(Dense(160, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(16, activation='softmax'))
          
sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
#adam = Adam(lr=1e-4)
# model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Using TensorFlow backend.


In [26]:
Lable = sample_lable_array[:,-1] -1 ## 取出标签
lable_ = keras.utils.to_categorical(Lable,num_classes=16)

In [27]:
Data.shape

(10249, 5, 5, 10)

In [28]:
data_train, data_test, lable_train, lable_test = train_test_split(Data,lable_,test_size=0.9) ## 分割训练测试集合

In [29]:
data_train.shape

(1024, 5, 5, 10)

In [30]:
data_test.shape

(9225, 5, 5, 10)

In [31]:
lable_train.shape

(1024, 16)

In [32]:
lable_test.shape

(9225, 16)

In [33]:
#### --------------------------------放入云服务器计算-----------------------------------------######
print('Training ------------------------')
model.fit(data_train, lable_train, batch_size=32, epochs=100)
print('Tseting -------------------------')
score = model.evaluate(data_test, lable_test, batch_size=32)
print('\ntest loss: ', score[0])
print('\ntest accuracy', score[1])

Training ------------------------
Epoch 1/100
1024/1024 [==============================] - 0s 422us/step - loss: 2.7162 - acc: 0.1377
Epoch 2/100
1024/1024 [==============================] - 0s 147us/step - loss: 2.3495 - acc: 0.2432
Epoch 3/100
1024/1024 [==============================] - 0s 158us/step - loss: 1.9438 - acc: 0.4102
Epoch 4/100
1024/1024 [==============================] - 0s 140us/step - loss: 1.6218 - acc: 0.4746
Epoch 5/100
1024/1024 [==============================] - 0s 142us/step - loss: 1.3659 - acc: 0.5205
Epoch 6/100
1024/1024 [==============================] - 0s 155us/step - loss: 1.2259 - acc: 0.5508
Epoch 7/100
1024/1024 [==============================] - 0s 149us/step - loss: 1.1443 - acc: 0.5820
Epoch 8/100
1024/1024 [==============================] - 0s 137us/step - loss: 1.0522 - acc: 0.5791
Epoch 9/100
1024/1024 [==============================] - 0s 144us/step - loss: 1.0055 - acc: 0.6025
Epoch 10/100
1024/1024 [==============================] - 0s 141us

1024/1024 [==============================] - 0s 143us/step - loss: 0.0034 - acc: 1.0000
Epoch 82/100
1024/1024 [==============================] - 0s 141us/step - loss: 0.0034 - acc: 1.0000
Epoch 83/100
1024/1024 [==============================] - 0s 146us/step - loss: 0.0032 - acc: 1.0000
Epoch 84/100
1024/1024 [==============================] - 0s 141us/step - loss: 0.0031 - acc: 1.0000
Epoch 85/100
1024/1024 [==============================] - 0s 159us/step - loss: 0.0030 - acc: 1.0000
Epoch 86/100
1024/1024 [==============================] - 0s 144us/step - loss: 0.0029 - acc: 1.0000
Epoch 87/100
1024/1024 [==============================] - 0s 153us/step - loss: 0.0028 - acc: 1.0000
Epoch 88/100
1024/1024 [==============================] - 0s 144us/step - loss: 0.0028 - acc: 1.0000
Epoch 89/100
1024/1024 [==============================] - 0s 138us/step - loss: 0.0027 - acc: 1.0000
Epoch 90/100
1024/1024 [==============================] - 0s 154us/step - loss: 0.0026 - acc: 1.0000
Epo